In [50]:
import re, string 
import nltk
import random 

from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize 

In [52]:
nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(tweet_tokens[0])

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


In [8]:
positive_tweets = twitter_samples.strings('positive_tweets.json') 
negative_tweets = twitter_samples.strings('negative_tweets.json') 
text = twitter_samples.strings('tweets.20150430-223406.json') 
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]
print(tweet_tokens[0])

#FollowFriday


In [9]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [10]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))

    return lemmatized_sentence 

print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [37]:

# ________ Something wrong with this function __________
def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|"\"(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        
    lemmatizer = WordNetLemmatizer()
    token = lemmatizer.lemmatize(token, pos)
         
    if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
        cleaned_tokens.append(token.lower())

    return cleaned_tokens


In [40]:
stop_words = stopwords.words('english')

print(remove_noise(tweet_tokens[0], stop_words))

[':)']


In [34]:
stop_words = stopwords.words('english') 

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))


In [36]:
 print(positive_tweet_tokens[500]) 
 print(positive_cleaned_tokens_list[500])

['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
[]


In [41]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token 

all_pos_words = get_all_words(positive_cleaned_tokens_list)

In [43]:
freq_dist_pos = FreqDist(all_pos_words)

print(freq_dist_pos.most_common(10))


[(':)', 1894), (':-)', 389), (':d', 388), (':p', 99), ('x', 44), ('xx', 38), ('...', 15), ('#wsalelove', 11), ('<3', 11), ('#ceo1month', 9)]


In [44]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens) 
        
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)


In [46]:

positive_dataset = [(tweet_dict, "Positive")for tweet_dict in positive_tokens_for_model] 
negative_dataset = [(tweet_dict, "Negative")for tweet_dict in negative_tokens_for_model] 
dataset = positive_dataset + negative_dataset 

random.shuffle(dataset) 

train_data = dataset[:7000]
test_data = dataset[7000:]


In [48]:
classifier = NaiveBayesClassifier.train(train_data) 

print("Accuracy is:", classify.accuracy(classifier, test_data)) 
print(classifier.show_most_informative_features(10))

Accuracy is: 0.7836666666666666
Most Informative Features
                      :) = True           Positi : Negati =    886.2 : 1.0
              #wsalelove = True           Positi : Negati =      3.8 : 1.0
                     lol = True           Negati : Positi =      3.0 : 1.0
                      xx = True           Positi : Negati =      2.8 : 1.0
                      .. = True           Negati : Positi =      2.6 : 1.0
                       😊 = True           Positi : Negati =      2.3 : 1.0
                       💜 = True           Positi : Negati =      2.3 : 1.0
                       ” = True           Negati : Positi =      2.3 : 1.0
                     fun = True           Negati : Positi =      2.3 : 1.0
                      :( = None           Positi : Negati =      2.0 : 1.0
None


In [53]:
custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."
custom_tokens = remove_noise(word_tokenize(custom_tweet)) 

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative


In [56]:
custom_tweet = 'Congrats #SportStar on your 7th best goal from last season winning goal of the year :) #Baller #Topbin #oneofmanyworldies'
custom_tokens = remove_noise(word_tokenize(custom_tweet)) 

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative
